In [0]:
import sys, platform, os
from matplotlib import pyplot as plt
import numpy as np
import math
from scipy.interpolate import *
import pandas as pd
from matplotlib import rc
from mpl_toolkits.mplot3d import Axes3D
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
cd /gdrive/My\ Drive/data

/gdrive/My Drive/data


In [0]:
pip install catboost

In [0]:
pip install pandas

In [0]:
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import gc
import os
import matplotlib.pyplot as plt
import seaborn as sns 
import lightgbm as lgb
from catboost import Pool, CatBoostClassifier
import itertools
import pickle, gzip
import glob
from sklearn.preprocessing import StandardScaler


In [0]:
train = pd.read_csv('plasticc_train_metadata.csv')
test = pd.read_csv('plasticc_test_metadata.csv')

In [0]:
train;

In [0]:
train_light = pd.read_csv('plasticc_train_lightcurves.csv')
# test_light_2 = pd.read_csv('plasticc_test_lightcurves_02.csv')
# test_light_3 = pd.read_csv('plasticc_test_lightcurves_03.csv')
# test_light_4 = pd.read_csv('plasticc_test_lightcurves_04.csv')

In [0]:
h = train_light['object_id']
id_code = h.unique()
len(id_code)

7848

In [0]:

train_light['flux_ratio_sq'] = np.power(train_light['flux'] / train_light['flux_err'], 2.0)
train_light['flux_by_flux_ratio_sq'] = train_light['flux'] * train_light['flux_ratio_sq']


aggs = {
    'mjd': ['min', 'max', 'size'],
    'passband': ['min', 'max', 'mean', 'median', 'std'],
    'flux': ['min', 'max', 'mean', 'median', 'std','skew'],
    'flux_err': ['min', 'max', 'mean', 'median', 'std','skew'],
    'flux_ratio_sq':['sum','skew'],
    'flux_by_flux_ratio_sq':['sum','skew'],
}

agg_train = train_light.groupby('object_id').agg(aggs)

new_columns = [
    k + '_' + agg for k in aggs.keys() for agg in aggs[k]
]
agg_train.columns = new_columns
agg_train['mjd_diff'] = agg_train['mjd_max'] - agg_train['mjd_min']
agg_train['flux_diff'] = agg_train['flux_max'] - agg_train['flux_min']
agg_train['flux_dif2'] = (agg_train['flux_max'] - agg_train['flux_min']) / agg_train['flux_mean']
agg_train['flux_w_mean'] = agg_train['flux_by_flux_ratio_sq_sum'] / agg_train['flux_ratio_sq_sum']
agg_train['flux_dif3'] = (agg_train['flux_max'] - agg_train['flux_min']) / agg_train['flux_w_mean']
# agg_train['object_id'] = id_code

# del agg_train['mjd_max'], agg_train['mjd_min']
agg_train.head()


,mjd_min,mjd_max,mjd_size,passband_min,passband_max,passband_mean,passband_median,passband_std,flux_min,flux_max,flux_mean,flux_median,flux_std,flux_skew,flux_err_min,flux_err_max,flux_err_mean,flux_err_median,flux_err_std,flux_err_skew,flux_ratio_sq_sum,flux_ratio_sq_skew,flux_by_flux_ratio_sq_sum,flux_by_flux_ratio_sq_skew,mjd_diff,flux_diff,flux_dif2,flux_w_mean,flux_dif3
object_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
615,59750.4229,60624.2132,352,0,5,2.457386,2.0,1.720797,-1100.440063,660.626343,-123.096998,-89.477524,394.109851,-0.349540,2.130510,12.845472,4.482743,3.835269,1.744747,1.623740,2.929669e+06,0.812722,-9.601766e+08,-1.414322,873.7903,1761.066406,-14.306331,-327.742307,-5.373326
713,59825.2600,60674.0798,350,0,5,2.400000,2.0,1.746056,-14.735178,14.770886,-1.423351,-0.873033,6.471144,0.014989,0.639458,9.115748,2.359620,1.998217,1.509888,1.633246,5.886068e+03,3.439423,-2.875087e+04,-3.454554,848.8198,29.506064,-20.730002,-4.884564,-6.040676
730,59798.3205,60652.1660,330,0,5,2.336364,2.0,1.758750,-19.159811,47.310059,2.267434,0.409172,8.022239,3.177854,0.695106,11.281384,2.471061,1.990851,1.721134,1.823726,4.124452e+03,5.480405,1.046502e+05,5.989138,853.8455,66.469870,29.315018,25.373110,2.619697
745,59770.3662,60624.0722,351,0,5,2.378917,2.0,1.747328,-15.494463,220.795212,8.909206,1.035895,27.558208,4.979826,0.567170,55.892746,2.555576,1.819875,3.537324,10.741655,9.416165e+04,9.611274,1.439125e+07,11.141069,853.7060,236.289675,26.521968,152.835617,1.546038
1124,59750.4229,60624.2132,352,0,5,2.457386,2.0,1.720797,-16.543753,143.600189,7.145702,1.141288,20.051722,4.406298,0.695277,11.383690,2.753004,2.214854,1.933837,1.794938,3.432418e+04,7.868462,3.015599e+06,7.908174,873.7903,160.143942,22.411225,87.856390,1.822792


In [0]:
# gc.enable()
train_light['flux_ratio_sq'] = np.power(train_light['flux'] / train_light['flux_err'], 2.0)
train_light['flux_by_flux_ratio_sq'] = train_light['flux'] * train_light['flux_ratio_sq']

aggs = {
    'mjd': ['min', 'max', 'size'],
    'passband': ['min', 'max', 'mean', 'median', 'std'],
    'flux': ['min', 'max', 'mean', 'median', 'std','skew'],
    'flux_err': ['min', 'max', 'mean', 'median', 'std','skew'],
    'flux_ratio_sq':['min', 'max', 'mean', 'median', 'std','skew'],
    'flux_by_flux_ratio_sq':['min', 'max', 'mean', 'median', 'std','skew'],
}

agg_train = train_light.groupby('object_id').agg(aggs)
new_columns = [
    k + '_' + agg for k in aggs.keys() for agg in aggs[k]
]

agg_train['mjd_diff'] = agg_train['mjd_max'] - agg_train['mjd_min']
agg_train['flux_diff'] = agg_train['flux_max'] - agg_train['flux_min']
agg_train['flux_dif2'] = (agg_train['flux_max'] - agg_train['flux_min']) / agg_train['flux_mean']
agg_train['flux_w_mean'] = agg_train['flux_by_flux_ratio_sq_sum'] / agg_train['flux_ratio_sq_sum']
agg_train['flux_dif3'] = (agg_train['flux_max'] - agg_train['flux_min']) / agg_train['flux_w_mean']
agg_train['object_id'] = agg_train['object_id']



new_columns = agg_train.columns
agg_train.head()


KeyError: ignored

In [0]:

new_columns1 = ['object_id','flux_u_1','flux_u_2','flux_u_3','flux_u_4','flux_u_5','flux_u_6','flux_u_7','flux_u_8','flux_u_9','flux_u_10','flux_u_11','flux_u_12','flux_u_13','flux_u_14','flux_u_15','flux_u_16','flux_u_17','flux_u_18','flux_u_19','flux_u_20',
               'flux_g_1','flux_g_2','flux_g_3','flux_g_4','flux_g_5','flux_g_6','flux_g_7','flux_g_8','flux_g_9','flux_g_10','flux_g_11','flux_g_12','flux_g_13','flux_g_14','flux_g_15','flux_g_16','flux_g_17','flux_g_18','flux_g_19','flux_g_20',
               'flux_r_1','flux_r_2','flux_r_3','flux_r_4','flux_r_5','flux_r_6','flux_r_7','flux_r_8','flux_r_9','flux_r_10','flux_r_11','flux_r_12','flux_r_13','flux_r_14','flux_r_15','flux_r_16','flux_r_17','flux_r_18','flux_r_19','flux_r_20',
                'flux_i_1','flux_i_2','flux_i_3','flux_i_4','flux_i_5','flux_i_6','flux_i_7','flux_i_8','flux_i_9','flux_i_10','flux_i_11','flux_i_12','flux_i_13','flux_i_14','flux_i_15','flux_i_16','flux_i_17','flux_i_18','flux_i_19','flux_i_20',
                'flux_z_1','flux_z_2','flux_z_3','flux_z_4','flux_z_5','flux_z_6','flux_z_7','flux_z_8','flux_z_9','flux_z_10','flux_z_11','flux_z_12','flux_z_13','flux_z_14','flux_z_15','flux_z_16','flux_z_17','flux_z_18','flux_z_19','flux_z_20',
                'flux_y_1','flux_y_2','flux_y_3','flux_y_4','flux_y_5','flux_y_6','flux_y_7','flux_y_8','flux_y_9','flux_y_10','flux_y_11','flux_y_12','flux_y_13','flux_y_14','flux_y_15','flux_y_16','flux_y_17','flux_y_18','flux_y_19','flux_y_20'
               ]

In [0]:
new_column2 = ['object_id','flux_u_mean','flux_u_max','flux_u_min','flux_u_std',
              'flux_g_mean','flux_g_max','flux_g_min','flux_g_std',
               'flux_r_mean','flux_r_max','flux_r_min','flux_r_std',
               'flux_i_mean','flux_i_max','flux_i_min','flux_i_std',
               'flux_z_mean','flux_z_max','flux_z_min','flux_z_std',
               'flux_y_mean','flux_y_max','flux_y_min','flux_y_std',
              ]

In [0]:

# rah digar
favor_time = np.arange(0,1,0.05)
final = pd.DataFrame(data=[], index=range(len(id_code)), columns=new_columns1)
final2 = pd.DataFrame(data=[], index=range(len(id_code)), columns=new_column2)

for i,l in enumerate(id_code):
    n = 0
    h = []
    hh = []
    h.append(l)
    hh.append(l)
    temp = train_light[train_light['object_id']==l]
    a = 0
    for j in range(6):
        temp_flux = temp[temp['passband']==j]['flux'].values
        temp_time = temp[temp['passband']==j]['mjd'].values
        f = temp_flux
        t = temp_time

        norm_time = ((t-(np.min(t)))/((np.max(t))-(np.min(t))))
        
        m = temp_flux.mean()
        ma = temp_flux.max()
        mi = temp_flux.min()
        std = np.std(temp_flux)
        hh.append(m)
        hh.append(ma)
        hh.append(mi)
        hh.append(std)
        for g in range(4):
            final2.iloc[i][a] = hh[a]
            a += 1
        for k in range(len(favor_time)):
            
            time = favor_time[k]
            h.append(np.interp(time,norm_time,f))
            final.iloc[i][n] = h[n]
            n += 1

In [0]:
final.iloc[1][1]

7.267655

In [0]:
train_new = final.reset_index().merge(
        right=final2,
        how='outer',
        on='object_id')
train_new = train_new.drop('index',axis=1)
train_new.head()

,object_id,flux_u_1,flux_u_2,flux_u_3,flux_u_4,flux_u_5,flux_u_6,flux_u_7,flux_u_8,flux_u_9,flux_u_10,flux_u_11,flux_u_12,flux_u_13,flux_u_14,flux_u_15,flux_u_16,flux_u_17,flux_u_18,flux_u_19,flux_u_20,flux_g_1,flux_g_2,flux_g_3,flux_g_4,flux_g_5,flux_g_6,flux_g_7,flux_g_8,flux_g_9,flux_g_10,flux_g_11,flux_g_12,flux_g_13,flux_g_14,flux_g_15,flux_g_16,flux_g_17,flux_g_18,flux_g_19,...,flux_y_5,flux_y_6,flux_y_7,flux_y_8,flux_y_9,flux_y_10,flux_y_11,flux_y_12,flux_y_13,flux_y_14,flux_y_15,flux_y_16,flux_y_17,flux_y_18,flux_y_19,flux_y_20,flux_u_mean,flux_u_max,flux_u_min,flux_u_std,flux_g_mean,flux_g_max,flux_g_min,flux_g_std,flux_r_mean,flux_r_max,flux_r_min,flux_r_std,flux_i_mean,flux_i_max,flux_i_min,flux_i_std,flux_z_mean,flux_z_max,flux_z_min,flux_z_std,flux_y_mean,flux_y_max,flux_y_min,flux_y_std
0,615,6.87878,-78.4791,42.6624,15.4438,-11.7748,-38.9934,-66.212,-93.4306,-90.5787,-77.6889,55.1475,95.2943,7.52015,27.119,46.7179,66.3168,85.9157,105.515,103.595,-65.835,-816.434,-66.4261,-713.091,-357.937,-764.938,-789.759,-814.579,-839.4,-864.221,-904.656,-637.086,-209.76,524.026,291.282,58.5386,-174.205,-406.949,-1014.09,-472.936,...,-407.742,-409.051,-410.36,-411.669,-412.978,68.6292,238.424,-124.689,206.631,84.4604,-37.7102,-159.881,-282.051,-350.493,-139.976,NaN,-3.25455,125.183,-116.913,83.2758,-385.7,660.626,-1100.44,596.577,-134.147,611.985,-681.859,451.181,-121.104,445.737,-530.645,332.521,-55.9546,381.954,-422.185,289.277,-47.4498,378.188,-422.815,NaN
1,713,7.26766,2.29758,1.04186,2.3522,2.32569,2.29919,2.27268,2.24617,2.21966,6.55278,-0.936404,-4.0339,-4.86599,-5.69808,-6.53017,-7.36226,-8.19435,-9.02644,-9.49311,-7.88524,7.61504,1.68119,7.24454,-1.46977,0.159774,1.78932,3.41887,5.04842,6.67796,2.02739,-0.6147,-1.97275,-4.21861,-5.0633,-5.90799,-6.75268,-7.59737,-8.44206,-7.72867,...,-1.69305,-0.610715,0.47162,1.55396,2.63629,5.60521,-8.56117,-6.18017,5.56622,2.60171,-0.362796,-3.32731,-6.29182,-9.25633,-12.9155,NaN,-2.7204,14.5098,-14.7352,7.06252,-1.0198,9.12902,-11.7157,5.6611,-0.794238,10.529,-10.0679,5.71898,-0.986966,11.3303,-12.3946,6.39256,-0.900262,9.82793,-12.2868,6.34953,-1.79417,14.7709,-14.2112,NaN
2,730,-2.3422,-0.767737,-0.523871,1.57461,1.60646,1.63831,1.67016,1.702,1.73385,0.30867,-0.290452,0.743879,1.48515,1.25964,1.03414,0.80864,0.583139,0.357637,1.80529,1.2587,2.32085,-0.633561,-1.12116,0.365921,-0.630076,-1.10913,-1.58819,-2.06725,-2.54631,-1.35026,0.715463,0.177047,-1.72538,-1.10349,-0.481609,0.140276,0.762161,1.38405,-0.0705204,...,3.11844,0.506037,-2.10637,-4.71877,-7.33117,-8.53718,0.428103,-3.60391,-7.49714,2.95765,13.4124,23.8672,34.322,44.7768,31.7941,NaN,-0.0480804,5.94217,-3.45996,1.81613,0.141057,5.69311,-3.39308,1.78977,2.40087,20.9947,-2.84884,5.50577,3.23616,33.5721,-5.4358,8.11284,4.30873,41.16,-5.83631,10.6048,4.5394,47.3101,-19.1598,NaN
3,745,0.535577,-0.485759,-2.57514,3.39362,6.29948,9.20535,12.1112,15.0171,17.9229,7.37694,0.238028,2.98189,0.485179,-0.159304,-0.803787,-1.44827,-2.09275,-0.524377,0.513763,-0.166529,-0.699347,-0.737064,0.586795,1.51661,32.6001,69.5714,106.543,143.514,180.485,31.7675,5.31624,-0.844759,2.57066,2.34028,2.1099,1.87953,1.64915,0.633563,0.829156,...,25.1971,52.1346,79.0721,106.01,132.947,77.6374,24.199,11.2882,10.0719,6.79609,3.52028,0.244468,-3.03134,-4.51267,7.37099,NaN,1.79752,18.014,-3.87435,4.34396,5.71739,192.244,-3.61841,25.7318,9.71153,220.795,-2.15975,31.6714,14.4129,203.251,-4.94404,34.6541,13.1344,183.633,-15.4945,32.7725,10.7461,141.513,-10.2494,NaN
4,1124,-0.358299,0.19801,-1.61174,-1.03707,-0.462389,0.112287,0.686963,1.26164,0.937081,-0.701675,-1.13385,4.71946,0.424438,1.39648,2.36853,3.34057,4.31262,5.28466,4.25506,1.10055,0.01647,0.756483,-1.89932,-0.174303,-0.712919,0.214134,1.14119,2.06824,2.99529,2.17088,0.140304,1.13047,4.68437,11.2674,17.8504,24.4334,31.0164,33.0216,14.2942,...,-3.26156,-1.97049,-0.679413,0.611659,1.90273,-1.64962,-1.61352,1.67216,13.7213,33.088,52.4548,71.8216,91.1883,97.4949,19.2129,NaN,0.660948,5.33093,-6.8047,2.3

In [0]:
# full_train = pd.merge(train_new, agg_train, on='object_id')
full_train = agg_train.reset_index().merge(
        right=train_new,
        how='outer',
        on='object_id')
# full_train = train_new.drop('index',axis=1)

ValueError: ignored

In [0]:
"cd ../

/gdrive/My Drive


In [0]:
df = pd.read_csv('Full_Features_train.csv')
df = df.drop('Unnamed: 0',axis=1)

In [0]:
# Full = pd.merge(full_train,df)
feat = Full.columns
# for i in range(len(feat)):
#     f = feat[i]
#     print(f)


full = full_train.reset_index().merge(
        right=df,
        how='outer',
        on='object_id')

In [0]:
train_new.to_csv('Nn1.csv')

In [0]:
#############              for test set              ######################

In [0]:
 from sysimport sys

In [0]:
id_code = []
h2 = test_light_2['object_id']
h3 = test_light_3['object_id']
h4 = test_light_4['object_id']
id_code.append(h2.unique())
id_code.append(h3.unique())
id_code.append(h4.unique())

AttributeError: ignored

In [0]:
console.log(h2.concat(h3))

NameError: ignored

In [0]:
##################         load of clap             ########################

In [0]:
new_df = np.zeros(shape=(len(id_code),6,100))
for i,l in enumerate(id_code):
    #print(i,l)
    temp = train_light[train_light['object_id']==l]
    for j in range(6):
        temp_1 = temp[temp['passband']==j]['flux'].values
        for k in range(len(temp_1)):
            new_df[i,j,k] = temp_1[k]

KeyboardInterrupt: ignored

In [0]:
new_df_flux = np.zeros(shape=(len(id_code),6,100))
new_df_time = np.zeros(shape=(len(id_code),6,100))

for i,l in enumerate(id_code):
    #print(i,l)
    temp = train_light[train_light['object_id']==l]
    for j in range(6):
        temp_flux = temp[temp['passband']==j]['flux'].values
        temp_time = temp[temp['passband']==j]['mjd'].values
        
        for k in range(len(temp_flux)):
            new_df_flux[i,j,k] = temp_flux[k]
        for m in range(len(temp_time)):
            new_df_time[i,j,m] = temp_time[m]

In [0]:
# normalization proses


# norm_df_flux = np.zeros(shape=(len(id_code),6,100))
norm_df_time = np.zeros(shape=(len(id_code),6,100))


for i,l in enumerate(id_code):
    for j in range(6):
        f = new_df_flux[i,j,:]
        t = new_df_time[i,j,:]

#         norm_flux = ((f-(np.min(f)))/((np.max(f))-(np.min(f))))
        norm_time = ((t-(np.min(t)))/((np.max(t))-(np.min(t))))
        
#         for k in range(len(norm_flux)):
#             norm_df_flux[i,j,k] = norm_time[k]
        for m in range(len(norm_time)):
            norm_df_time[i,j,m] = norm_time[m]
        
        
        

In [0]:
#interpolation part
favor_time = np.arange(0,1,0.1)

interp_df_flux = np.zeros(shape=(len(id_code),6,100))

for i,l in enumerate(id_code):
    
    for j in range(6):
        
        f = new_df_flux[i,j,:]
        t = norm_df_time[i,j,:]
        
        for k in range(len(favor_time)):
            time = favor_time[k]
            interp_df_flux[i,j,k] = np.interp(time,t,f)
            
        

In [0]:

# ncol = []
# ncol = new_columns.values
# ncol.append(new_columns1)
# agg_train.columns = ncol


for j in range(6):
    for k in range(len(favor_time)):
        for m in range(len(new_columns1)):
            yy = interp_df_flux[:,j,k]
            print(yy)
            agg_train[new_columns1[m]] = yy
            print(agg_train[new_columns1[m]])
          

[   6.878784    7.267655   -2.3422   ...    7.376702    8.315109
 1036.600708]


NameError: ignored

In [0]:
export_csv = full_train2.to_csv('nazi_full.csv')

In [0]:
!pip install gatspy
# from gatspy import datasets, periodic




     |████████████████████████████████| 563kB 2.8MB/s 
  Created wheel for gatspy: filename=gatspy-0.3-cp36-none-any.whl size=43806 sha256=09f85a5ce1f41bd3ffd770646e7549229fb1d6ed46418722dfd7b954da9380d4
  Stored in directory: /root/.cache/pip/wheels/4f/8f/fa/0d7b250ef21828ca373b21f6b3b6ef0f2a0e3560b69c91e55d
Successfully built gatspy


In [0]:
from gatspy.periodic import LombScargleMultiband
from gatspy import datasets, periodic

In [0]:
lc_full  = train_light
lc_full.columns

Index(['object_id', 'mjd', 'passband', 'flux', 'flux_err', 'detected_bool'], dtype='object')

In [0]:
model = LombScargleMultiband(fit_period=True)
fitur_df = pd.DataFrame(data=[], index=range(len(id_code)), columns=['object_id','period'])
# we'll window the search range by setting minimums and maximums here
# but in general, the search range you want to evaluate will depend on the data
# and you will not be able to window like this unless you know something about
# the class of the object a priori
for i in range(len(id_code)):
    p =[]
    lc = lc_full[lc_full['object_id']==id_code[i]]
    t_min = max(np.median(np.diff(sorted(lc['mjd']))), 0.1)
    t_max = min(10., (lc['mjd'].max() - lc['mjd'].min())/2.)

    model.optimizer.set(period_range=(t_min, t_max), first_pass_coverage=5)
    model.fit(lc['mjd'], lc['flux'], dy=lc['flux_err'], filts=lc['passband'])
    period = model.best_period
    p.append(id_code[i])
    p.append(period)
    fitur_df.ix[i] = p
    print(f' has a period of {period} days')

Finding optimal frequency:
 - Estimated peak width = 0.00719
 - Using 5 steps per peak; omega_step = 0.00144
 - User-specified period range:  0.1 to 10
 - Computing periods at 43254 steps
Zooming-in on 5 candidate peaks:
 - Computing periods at 995 steps
 has a period of 0.3244994176047133 days
Finding optimal frequency:
 - Estimated peak width = 0.0074
 - Using 5 steps per peak; omega_step = 0.00148
 - User-specified period range:  0.1 to 10
 - Computing periods at 42018 steps


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


Zooming-in on 5 candidate peaks:
 - Computing periods at 1000 steps
 has a period of 0.9965878932095302 days
Finding optimal frequency:
 - Estimated peak width = 0.00736
 - Using 5 steps per peak; omega_step = 0.00147
 - User-specified period range:  0.1 to 10
 - Computing periods at 42267 steps
Zooming-in on 5 candidate peaks:
 - Computing periods at 995 steps
 has a period of 0.999535202267766 days
Finding optimal frequency:
 - Estimated peak width = 0.00736
 - Using 5 steps per peak; omega_step = 0.00147
 - User-specified period range:  0.1 to 10
 - Computing periods at 42260 steps
Zooming-in on 5 candidate peaks:
 - Computing periods at 1000 steps
 has a period of 0.9986820811466305 days
Finding optimal frequency:
 - Estimated peak width = 0.00719
 - Using 5 steps per peak; omega_step = 0.00144
 - User-specified period range:  0.1 to 10
 - Computing periods at 43254 steps
Zooming-in on 5 candidate peaks:
 - Computing periods at 995 steps
 has a period of 0.9949866033438874 days
Fin

KeyboardInterrupt: ignored

In [0]:
fitur_df.to_csv('nazi_priod.csv')

In [0]:
fitur_df.iloc[1070]

object_id    NaN
period       NaN
Name: 1070, dtype: object